<a href="https://colab.research.google.com/github/AbhinayReddyYarva/NLP-SpamSMSDetection-KaggleDataset/blob/master/ModelBuildEvaluateGPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building ML Classifiers: Basic Random Forest & Gradient Boosting models (Running in Google colab with TPU)

### Reading & cleaning text

In [0]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
import time
import datetime


nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation]) # Counting puntuations
    return round(count/(len(text) - text.count(" ")), 3)*100  # Percentage of punctuations

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation]) # removing puntuations and creating list
    tokens = re.split('\W+', text) # removing multiple special characters --> '\W+' ('\W' -> single special character)
    #tokens = re.findall('\w+', text)  # alternative to above statement (Split -> '\W+'; findall -> '\w+')
    text = [ps.stem(word) for word in tokens if word not in stopwords]  # stemmig the words
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
print(X_features.head())

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)
print(X_count_feat.head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
   body_len  punct%    0    1    2    3  ...  8098  8099  8100  8101  8102  8103
0       128     4.7  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
1        49     4.1  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
2        62     3.2  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
3        28     7.1  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
4       135     4.4  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0

[5 rows x 8106 columns]
   body_len  punct%  0  1  2  3  4  ...  8097  8098  8099  8100  8101  8102  8103
0       128     4.7  0  0  0  0  0  ...     0     0     0     0     0     0     0
1        49     4.1  0  0  0  0  0  ...     0     0     0     0     0     0     0
2        62     3.2  0  0  0  0  0  ...     0     0     0     0     0     0     0
3        28     7.1  0  0  0  0  0  ...     0     

### Exploring RandomForestClassifier Attributes & Hyperparameters

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto

### Exploing RandomForestClassifier through Cross-Validation

In [0]:
from sklearn.model_selection import KFold, cross_val_score

In [0]:
rf_kfold = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf_kfold, X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


array([0.97217235, 0.97396768, 0.9703504 , 0.95777179, 0.96855346])

### Exploring RandomForestClassifier through holdout set

In [0]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [0]:
rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

In [0]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10]

[(0.05171580149791864, 'body_len'),
 (0.04373805861312181, 7350),
 (0.031866135091081334, 5724),
 (0.03109825660341768, 2031),
 (0.02681957094281602, 3134),
 (0.023411880422090213, 392),
 (0.022780378296251957, 4796),
 (0.021606176276297056, 6285),
 (0.021532508930721037, 7218),
 (0.01928722545545524, 7027)]

In [0]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [0]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

Precision: 1.0 / Recall: 0.529 / Accuracy: 0.942


### Building a Grid Search

In [0]:
def train_RF(n_est, depth):
    rf1 = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model1 = rf.fit(X_train, y_train)
    y_pred1 = rf_model.predict(X_test)
    precision1, recall1, fscore1, support1 = score(y_test, y_pred1, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision1, 3), round(recall1, 3),
        round((y_pred1==y_test).sum() / len(y_pred1), 3)))

In [0]:
print(datetime.datetime.now())
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)
print(datetime.datetime.now())

2019-06-05 21:17:38.800092
Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.536 / Accuracy: 0.943
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.536 / Accuracy: 0.943
Est: 10 / Depth: 30 ---- Precision: 1.0 / Recall: 0.565 / Accuracy: 0.946
Est: 10 / Depth: None ---- Precision: 1.0 / Recall: 0.572 / Accuracy: 0.947
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.529 / Accuracy: 0.942
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.536 / Accuracy: 0.943
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.543 / Accuracy: 0.943
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.594 / Accuracy: 0.95
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.5 / Accuracy: 0.938
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.601 / Accuracy: 0.951
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.551 / Accuracy: 0.944
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.522 / Accuracy: 0.941
2019-06-05 21:17:53.000891


### Evaluating Random forest with GridSearchCV

In [0]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
rf2 = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf2, param, cv=5, n_jobs=-1)

In [0]:
start = time.time()
print(datetime.datetime.now())

gs_fit1 = gs.fit(X_features, data['label'])
pd.DataFrame(gs_fit1.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

2019-06-05 21:17:53.045977


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,15.031352,0.228887,0.174326,0.015621,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.980269,0.979335,0.973944,0.966757,0.971249,0.974313,0.005047,1
10,16.435477,0.314616,0.173929,0.004407,None,150,"{'max_depth': None, 'n_estimators': 150}",0.977578,0.977538,0.975741,0.966757,0.973944,0.974313,0.004008,1
3,1.157143,0.011932,0.073879,0.001593,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.977578,0.979335,0.975741,0.968553,0.969452,0.974133,0.004349,3
6,1.349629,0.011598,0.073353,0.000813,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.977578,0.977538,0.978437,0.959569,0.973944,0.973415,0.007092,4
8,29.419046,0.344261,0.256684,0.004135,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976682,0.976640,0.974843,0.968553,0.970350,0.973415,0.003351,4


In [0]:
end = time.time()
fit_time = (end - start)
print(fit_time)
print(datetime.datetime.now())

422.89899611473083
2019-06-05 21:24:55.945895


In [0]:
start = time.time()
print(datetime.datetime.now())

gs_fit2 = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit2.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

2019-06-05 21:24:55.955070


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
6,1.893950,0.233560,0.068530,0.003222,90,10,"{'max_depth': 90, 'n_estimators': 10}",0.977578,0.971249,0.973046,0.976640,0.967655,0.973235,0.003622,1
8,29.650184,0.385415,0.258950,0.013477,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976682,0.975741,0.973046,0.967655,0.970350,0.972696,0.003353,2
7,15.258343,0.213648,0.159749,0.001054,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.978475,0.975741,0.972147,0.967655,0.968553,0.972517,0.004133,3
11,31.564750,2.344075,0.264692,0.031661,None,300,"{'max_depth': None, 'n_estimators': 300}",0.976682,0.973944,0.974843,0.968553,0.968553,0.972517,0.003353,3
10,16.796552,0.235082,0.176762,0.008660,None,150,"{'max_depth': None, 'n_estimators': 150}",0.975785,0.973046,0.973944,0.968553,0.968553,0.971978,0.002931,5


In [0]:
end = time.time()
fit_time = (end - start)
print(fit_time)
print(datetime.datetime.now())

430.40967655181885
2019-06-05 21:32:06.365831


### Exploring GradientBoostingClassifier Attributes & Hyperparameters

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

['_SUPPORTED_LOSS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_check_initialized', '_check_params', '_clear_state', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_get_tags', '_init_state', '_is_initialized', '_make_estimator', '_raw_predict', '_raw_predict_init', '_required_parameters', '_resize_state', '_staged_raw_predict', '_validate_estimator', '_validate_y', 'apply', 'decision_function', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score

### Building a Grid Search for Gradient Boosting

In [0]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred2 = gb_model.predict(X_test)
    precision2, recall2, fscore2, train_support2 = score(y_test, y_pred2, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision2, 3), round(recall2, 3), 
        round((y_pred2==y_test).sum()/len(y_pred2), 3)))

In [0]:
start = time.time()
print(datetime.datetime.now())

for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)
            end = time.time()
            fit_time = (end - start)
            print(fit_time)
            print(datetime.datetime.now())
print(datetime.datetime.now())

2019-06-05 21:32:06.404302


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 3 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.876
30.335994720458984
2019-06-05 21:32:36.740292
Est: 50 / Depth: 3 / LR: 0.1 ---- Precision: 0.925 / Recall: 0.717 / Accuracy: 0.958
55.78804540634155
2019-06-05 21:33:02.192328
Est: 50 / Depth: 3 / LR: 1 ---- Precision: 0.886 / Recall: 0.79 / Accuracy: 0.961
74.34766364097595
2019-06-05 21:33:20.752341
Est: 50 / Depth: 7 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.877
143.74246335029602
2019-06-05 21:34:30.147090
Est: 50 / Depth: 7 / LR: 0.1 ---- Precision: 0.911 / Recall: 0.812 / Accuracy: 0.967
204.8179795742035
2019-06-05 21:35:31.222652
Est: 50 / Depth: 7 / LR: 1 ---- Precision: 0.868 / Recall: 0.812 / Accuracy: 0.961
248.13868927955627
2019-06-05 21:36:14.543344
Est: 50 / Depth: 11 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.007 / Accuracy: 0.877
352.121258020401
2019-06-05 21:37:58.525559
Est: 50 / Depth: 11 / LR: 0.1 ---- Precision: 0.891 / Recall: 0.826 / Accuracy: 0.966
446.6251826

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 15 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.876
622.8722188472748
2019-06-05 21:42:29.276504
Est: 50 / Depth: 15 / LR: 0.1 ---- Precision: 0.896 / Recall: 0.812 / Accuracy: 0.965
754.9626662731171
2019-06-05 21:44:41.367267
Est: 50 / Depth: 15 / LR: 1 ---- Precision: 0.87 / Recall: 0.826 / Accuracy: 0.963
788.8237068653107
2019-06-05 21:45:15.228009
Est: 100 / Depth: 3 / LR: 0.01 ---- Precision: 0.939 / Recall: 0.558 / Accuracy: 0.941
849.0492794513702
2019-06-05 21:46:15.453893
Est: 100 / Depth: 3 / LR: 0.1 ---- Precision: 0.923 / Recall: 0.783 / Accuracy: 0.965
893.4717948436737
2019-06-05 21:46:59.876090
Est: 100 / Depth: 3 / LR: 1 ---- Precision: 0.893 / Recall: 0.79 / Accuracy: 0.962
928.7434177398682
2019-06-05 21:47:35.147716
Est: 100 / Depth: 7 / LR: 0.01 ---- Precision: 0.938 / Recall: 0.652 / Accuracy: 0.952
1069.1125764846802
2019-06-05 21:49:55.517237
Est: 100 / Depth: 7 / LR: 0.1 ---- Precision: 0.928 / Recall: 0.841 / Accuracy: 0.972
117

### Evaluating Gradient Boosting with GridSearchCV

In [0]:
#from sklearn.model_selection import GridSearchCV

gb2 = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb2, param, cv=5, n_jobs=-1)

In [0]:
start = time.time()
print(datetime.datetime.now())

cv_fit = clf.fit(X_features, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

2019-06-05 22:39:41.718817


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [0]:
end = time.time()
fit_time = (end - start)
print(fit_time)
print(datetime.datetime.now())

In [0]:
start = time.time()
print(datetime.datetime.now())

cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

In [0]:
end = time.time()
fit_time = (end - start)
print(fit_time)
print(datetime.datetime.now())